# MNE Sample Dataset: Auditory and Visual Evoked Responses

This notebook uses the MNE-Python sample dataset, which contains MEG and EEG recordings from a single subject who was presented with auditory and visual stimuli.

## Dataset Details:
- **Subject**: A healthy adult volunteer
- **Recording**: 204 planar gradiometers, 102 magnetometers, and 60 EEG channels
- **Paradigm**: Simple auditory and visual stimulation with left/right conditions
- **Trials**: Approximately 100 epochs each for the different conditions

## Experimental Conditions in Detail:

### Auditory Experiment
The auditory experiment presents tone beeps to either the left or right ear:
- **Stimulus type**: Pure tone beeps
- **Frequency**: 1 kHz (1000 Hz)
- **Duration**: 50 ms
- **Intensity**: 60 dB sound pressure level
- **Presentation**: 
  - Left ear (Event code: '1' or 'Auditory/Left')
  - Right ear (Event code: '2' or 'Auditory/Right')
- **Inter-stimulus interval**: Variable, around 1-1.5 seconds
- **Neural response**: 
  - Both conditions activate the auditory cortices in the temporal lobes
  - Stronger contralateral activation (left ear stimuli activate right hemisphere more, and vice versa)
  - Peak responses typically occur around 100 ms (N100/M100) and 200 ms (P200/M200) post-stimulus

### Visual Experiment
The visual experiment involves presenting checkerboard patterns to the left and right visual fields separately:
- **Stimulus type**: Black and white checkerboard pattern
- **Presentation**: 
  - Left visual field (Event code: '3' or 'Visual/Left')
  - Right visual field (Event code: '4' or 'Visual/Right')
- **Duration**: 50 ms
- **Visual angle**: Approximately 4 degrees, presented about 5 degrees to the left or right of fixation
- **Contrast**: High contrast (black/white)
- **Inter-stimulus interval**: Variable, around 1-1.5 seconds
- **Neural response**: 
  - Left visual field stimuli primarily activate the right occipital cortex
  - Right visual field stimuli primarily activate the left occipital cortex
  - Characteristic components appear around 100 ms (P100/M100) and 170 ms (N170/M170) post-stimulus
  - This contralateral activation pattern allows for clear demonstration of the retinotopic organization of the visual system

## Research Applications:
This dataset is ideal for demonstrating:
- Basic preprocessing of MEG/EEG data
- Evoked response analysis 
- Source localization techniques
- Comparison between different sensor types
- Time-frequency analysis
- Contralateral organization of both auditory and visual systems
- Differences in spatial and temporal patterns between auditory and visual processing
- Hemispheric specialization and lateralization effects

The data follows the standard organization used in MNE-Python, making it easy to access events, 
raw recordings, and metadata through the MNE API.


In [ ]:
%matplotlib qt

import mne
from mne.datasets import sample
import matplotlib.pyplot as plt
import numpy as np

### Load the data

In [58]:
# Load MNE sample data
data_path = sample.data_path()
raw_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
event_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw-eve.fif'

# Read raw EEG data
raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw.pick_types(meg=False, eeg=True, stim=True)

# Bandpass filter the raw data (1-30 Hz)
raw.filter(1., 30., fir_design='firwin')

Opening raw data file /home/koutras/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...


    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1983 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


<Raw | sample_audvis_raw.fif, 68 x 166800 (277.7 s), ~89.5 MiB, data loaded>

### Make epochs from the data and plot them.
First we define the different events. These are the Auditory/Left, Auditory/Right, Visual/Left, Visual/Right

In [59]:
# Read events
events = mne.read_events(event_fname)

# Create epochs for auditory stimuli (event_id = 1)
event_id = {'Auditory/Left': 1,
            'Auditory/Right':2,
            'Visual/Left':3,
            'Visual/Right':4}
            


tmin, tmax = -0.2, 0.5  # define epoch interval
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, baseline=(None, 0), preload=True)

# Plot epochs to visually inspect trials
epochs.plot(n_epochs=10, title='Auditory Stimulus Epochs', scalings='auto')

Not setting metadata
289 matching events found


Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 289 events and 421 original time points ...
0 bad epochs dropped


qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


<mne_qt_browser._pg_figure.MNEQtBrowser(0x58f6da6b0300) at 0x78946d5fcf80>

### Make and plot the evoked response potentials. These are the mean values  of all epochs for the different channels.

Looking at these ERP plots for left and right auditory stimulation, we can identify several important features:
1. **Overall waveform morphology**:
   - Both conditions show the classic auditory evoked potential components
   - Clear negative peak around 100ms (N100)
   - Prominent positive peak around 200ms (P200)
   - Later components visible after 300ms

2. **N100 component**:
   - In both conditions, there's a pronounced negative deflection around 100-120ms
   - The N100 appears to have slightly different amplitudes between conditions
   - In the left auditory condition (top), the N100 peaks at approximately -10μV
   - In the right auditory condition (bottom), similar amplitude but slight differences in the distribution across channels

3. **P200 component**:
   - Visible positive peak around 180-220ms in both conditions
   - In the left auditory condition, P200 appears to have greater amplitude in some channels
   - Channel variation suggests different spatial distributions of this component

4. **Channel variation**:
   - The colored lines represent different EEG channels
   - The variation in amplitude across channels reflects the spatial distribution of activity
   - Some channels show stronger responses than others, consistent with the topographic maps you showed previously

5. **Hemispheric differences**:
   - While it's difficult to identify specific channels from the plot, the variation in waveforms between conditions likely corresponds to the contralateral dominance pattern
   - Channels over the right hemisphere would show stronger responses for left ear stimulation and vice versa

6. **Post-stimulus variability**:
   - After about 300ms, there's considerable variability between channels
   - This reflects more individualized processing after the initial sensory components

These ERP plots complement the topographic maps by showing the precise timing of the auditory responses, while also illustrating how the response varies across different scalp locations.

In [60]:
evoked_all = epochs.average()
# Plot ERP (Evoked response)
evoked_left = epochs["Auditory/Left"].average()
evoked_left.plot()
plt.title('Evoked Response - Left')
evoked_right = epochs["Auditory/Right"].average()
evoked_right.plot()
plt.title('Evoked Response - Right')

plt.show()

### Next we plot the topographic plots for both cases at different time points after the stimuli.
Note the topography in both cases in ~N100 and ~P200.

### For the N100 component (around 94-117ms):

#### Left auditory stimulation (top row):
1. Strong negative (blue) activity predominantly in the **right** hemisphere.
2. This negativity is centered in the **right fronto-central/temporal regions**
3. **The contralateral nature (left ear → right hemisphere) is clearly visible**

#### Right auditory stimulation (bottom row):
1. Strong negative (blue) activity predominantly in the **left** hemisphere
2. This negativity is centered in the **left fronto-central/temporal regions**
3. **Again showing the expected contralateral activation pattern**

### For the P200 component (around 183-228ms):

#### Left auditory stimulation (top row):
1. Strong positive (red) activity that appears most prominently in **central and right** hemisphere regions
2. The positivity peaks around 206-228ms
3. **Shows a broader, more diffuse distribution compared to the N100**

#### Right auditory stimulation (bottom row):
1. Strong positive (red) activity that is strongest in **central and left** hemisphere regions
2. Similar timing to the left auditory P200
3. **Again demonstrates the contralateral dominance pattern, though less pronounced than in the N100**

### Key observations:

1. The N100 shows stronger lateralization (more pronounced contralateral activation) than the P200
2. The P200 has a more central and diffuse distribution, suggesting more bilateral processing at this later stage
3. There's a clear polarity inversion from N100 (negative) to P200 (positive)
4. The timing progression clearly shows the sequential processing stages of auditory information

These topographic differences reflect the neurophysiological principle of contralateral dominance in auditory processing pathways, where stimuli presented to one ear produce stronger responses in the opposite hemisphere, particularly at earlier processing stages like the N100.

In [61]:
times = np.linspace(0.05, 0.25, 10)
evoked_left.plot_topomap(times=times, colorbar=True)
evoked_right.plot_topomap(times=times, colorbar=True)

plt.show()

### Joint plots

This combined visualization provides an excellent integration of the ERP waveforms with topographic maps at key time points. Looking at these joint plots, we can make several important observations:

For Left Auditory Stimulation (top):
1. **N100 component (0.095s)**:
   - Strong negative (blue) activity predominantly in the right hemisphere
   - The topographic map shows clear contralateral activation (left ear → right hemisphere)
   - The ERP waveform shows this as a pronounced negative deflection across multiple channels

2. **P200 component (0.205s)**:
   - Clear positive (red) activity with a more central distribution
   - The topography shows stronger activation in central and right regions
   - The ERP shows this as a positive deflection following the N100

3. **Late component (0.448s)**:
   - Positive activity with a frontal and right hemisphere dominance
   - Represents later cognitive processing of the auditory stimulus

For Right Auditory Stimulation (bottom):
1. **N100 component (0.088s)**:
   - Strong negative (blue) activity predominantly in the left hemisphere
   - Clear contralateral activation (right ear → left hemisphere)
   - Similar latency to the left auditory N100, but with different spatial distribution

2. **Intermediate component (0.163s)**:
   - A transitional stage showing positive central activity with negative peripheral activity
   - This time point differs from the top plot, highlighting potential differences in processing speed

3. **P200 component (0.206s)**:
   - Positive (red) activity with stronger representation in central and left regions
   - Similar latency to the left auditory P200, but with different spatial distribution

Key comparative observations:
1. **Lateralization effects**:
   - Both conditions clearly demonstrate the principle of contralateral dominance in auditory processing
   - The N100 shows stronger lateralization than the P200 in both conditions

2. **Temporal differences**:
   - Slight timing differences in component peaks between conditions
   - The right auditory condition shows an additional topography at 0.163s not highlighted in the left condition

3. **Processing sequence**:
   - Both conditions demonstrate the classic progression from early sensory processing (N100) to later perceptual/cognitive stages (P200 and beyond)
   - The detailed evolution of the spatial distribution over time provides insight into how auditory information flows through brain networks

These visualizations effectively illustrate the neurophysiological principles of auditory processing, showing both the temporal dynamics (from the ERP waveforms) and spatial distribution (from the topographic maps) of brain activity during lateralized auditory stimulation.

In [62]:
evoked_left.plot_joint()
evoked_right.plot_joint()

plt.show()

No projector specified for this dataset. Please consider the method self.add_proj.
No projector specified for this dataset. Please consider the method self.add_proj.


### Image plots

These images show EEG channel by time heatmaps for the left auditory (top) and right auditory (bottom) conditions. This visualization format is particularly useful for seeing patterns across all channels simultaneously.

For the Left Auditory Stimulation (top):
1. **N100 component** (~0.1s):
   - Strong negative activity (dark blue) appears across multiple channels
   - This negativity is more prominent in certain channel ranges (particularly channels 10-20), which likely correspond to right hemisphere electrodes
   - The timing aligns with the N100 component we observed in previous visualizations

2. **P200 component** (~0.2s):
   - Clear positive activity (red) follows the N100
   - This positive response appears distributed across channels
   - Shows a distinct temporal pattern across different channel groupings

3. **Channel distribution**:
   - Different channels show varying response patterns
   - Some channel clusters show stronger responses than others, reflecting the spatial organization of auditory processing

For the Right Auditory Stimulation (bottom):
1. **N100 component** (~0.1s):
   - Strong negative activity (dark blue) but with a different distribution across channels
   - The strongest negativity appears to be in different channel ranges compared to the left auditory condition, likely reflecting left hemisphere dominance

2. **P200 component** (~0.2s):
   - Positive activity (red/orange) following the N100
   - The distribution across channels differs from the left auditory condition

3. **Later components**:
   - Both conditions show additional oscillatory patterns after 0.3s
   - The patterns differ between conditions, suggesting different later processing

Key comparative observations:
1. **Channel differences**:
   - The distribution of activity across channels differs between conditions
   - This reflects the contralateral processing we saw in the topographic maps

2. **Temporal sequence**:
   - Both conditions show the classic N100-P200 sequence
   - The timing of components is similar, but their distribution across channels differs

3. **Channel clustering**:
   - The activity appears to cluster in different channel groups between conditions
   - This clustering likely corresponds to the lateralized activation patterns observed in the topographic maps

These heatmaps provide an excellent overview of how the auditory responses vary across both time and space (channels), further supporting the lateralized processing patterns we've observed in the previous visualizations.

In [63]:
evoked_left.plot_image()
evoked_right.plot_image()

plt.show()

### Compare Evoked

These plots provide valuable comparative information between left and right auditory conditions:

On the left plot (Mean evoked response):
1. **N100 differences**:
   - The blue line (auditory_left) shows a stronger negative peak around 0.1s compared to the orange line (auditory_right)
   - The left auditory N100 appears to peak slightly earlier and with greater amplitude
   - This indicates differences in early sensory processing depending on which ear was stimulated

2. **P200 differences**:
   - Left auditory (blue) shows a stronger positive deflection around 0.2s
   - The morphology and timing of the P200 differs between conditions
   - Left auditory P200 appears to have higher amplitude

3. **Overall waveform differences**:
   - The conditions show different oscillatory patterns throughout the time course
   - Pre-stimulus activity also differs between conditions
   - Later components (after 0.3s) show distinct patterns

On the right plot (Global Field Power/GFP):
1. **Overall signal strength**:
   - GFP measures the spatial standard deviation across electrodes at each time point, representing the overall strength of the response regardless of topography
   - Both conditions show a prominent peak around 0.1s, corresponding to the N100 timeframe
   - The left auditory condition (blue) shows consistently higher GFP

2. **Component strength**:
   - The N100 component shows the highest GFP for both conditions, indicating this is when neural activity is most synchronized and strongest
   - Left auditory stimulation produces greater overall field strength than right auditory
   - This suggests potentially stronger or more synchronized neural responses for left ear stimulation

3. **Temporal dynamics**:
   - Both conditions show similar timing of major GFP peaks
   - The relative difference in GFP magnitude remains fairly consistent throughout the epoch
   - Secondary peaks after 0.2s show more variability between conditions

These plots highlight important asymmetries in auditory processing:
- The stronger response for left auditory stimulation (in both mean evoked and GFP) suggests potential right hemisphere dominance for auditory processing
- The timing differences in components point to potentially different processing speeds for left versus right ear stimulation
- The GFP differences indicate that not only does the spatial distribution of activity differ between conditions (as seen in the topographic maps), but also the overall strength of the neural response

In [64]:
evokeds = dict(auditory_left=evoked_left, auditory_right=evoked_right)

for combine in ("mean", "gfp"):
    mne.viz.plot_compare_evokeds(evokeds,combine=combine)
plt.show()

combining channels using "mean"
combining channels using "mean"
combining channels using GFP (eeg channels)
combining channels using GFP (eeg channels)


/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


### Next steps

You can analyze the visual stimulation data from the MNE sample dataset, I'd recommend focusing on these key aspects:

## Analysis Guidance for Visual Stimulation Data

### Key Time Points to Examine:
1. **P100 (around 80-120ms)**: The first major positive component, representing early visual processing
2. **N170 (around 150-190ms)**: A prominent negative component associated with visual object processing
3. **P300 (around 250-350ms)**: Later positive component reflecting higher-order cognitive processing

### Topographical Areas to Focus On:
1. **Occipital regions**: Primary visual areas (back of the head) for early visual responses
2. **Lateralization patterns**: 
   - Left visual field stimuli → activity predominantly in right occipital areas
   - Right visual field stimuli → activity predominantly in left occipital areas
3. **Ventral visual stream**: Flow of activation from occipital to temporal regions

### Specific Analyses to Assign:

1. **Compare topographies between left and right visual field conditions**:
   - Look for the contralateral organization (more pronounced than in auditory)
   - Observe how the earliest component (P100) shows strongest contralateral activity
   - Note how later components become more bilaterally distributed

2. **Examine ERP waveforms from occipital electrodes**:
   - Compare electrodes over left vs. right occipital regions
   - Look for amplitude differences between contralateral and ipsilateral responses

3. **Conduct PLV analysis**:
   - Focus on connectivity between occipital regions and other areas
   - Compare left vs. right visual field connectivity patterns
   - Look for differences in the timing of increased connectivity

4. **Create difference maps**:
   - Generate left-right visual field contrast topographies
   - Identify time points with maximum lateralization effects

5. **Compare GFP between conditions**:
   - Look for differences in overall response strength
   - Identify peak timing differences between conditions

The visual paradigm should show even clearer contralateral organization than the auditory data, making it an excellent demonstration of the retinotopic organization of the visual system. The occipital focus of activity (versus temporal for auditory) provides a good contrast for understanding sensory processing across different modalities.